In [2]:
import numpy as np
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation.quality_control import metrics, visualization

ModuleNotFoundError: No module named 'timagetk'

In [ ]:
v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

pred_paths = v.data_path + 'Gr1/Segmentation/Nuclei/QC_CROP/'
gt_path = (
        v.data_path 
        + 'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT.nii.gz'
)

gt = imaging.read_image(gt_path, axes='XYZ')

In [ ]:
from itertools import permutations


steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(0.1, .8, 8), 1))
steps_mandatory = ['isotropy', 'normalization']
steps_optional = [
    'bilateral', 
    'anisodiff'
]

steps_permutations = list(permutations(steps_optional))

pipelines_dict = {}

# Create pipelines for each type and threshold
for step in steps_type:
    for thr in steps_thr:
        for i, perm in enumerate(steps_permutations):
            # Create the pipeline with mandatory steps
            pipeline = steps_mandatory + list(perm)

            # Store the pipeline along with the step type and threshold
            pipeline_key = f'pipeline_{step}_{i}_thr_{thr}'
            pipelines_dict[pipeline_key] = {
                'pipeline': pipeline,
                'type': step,
                'threshold': thr
            }

# Print the created pipelines
for name, config in pipelines_dict.items():
    print(f"{name}: {config['pipeline']}, Type: {config['type']}, Threshold: {config['threshold']}")
    
print('Total pipelines:', len(pipelines_dict))

In [ ]:
bar = LoadingBar(len(pipelines_dict))

results = []
for name, config in pipelines_dict.items():
    print(f"\nPipeline: {name}")
    
    file_name = f'20190208_E2_DAPI_decon_0.5_crop_{name}.nii.gz'
    
    out_path = pred_paths + 'Stats/' + file_name
    pred = imaging.read_image(pred_paths + file_name, axes='XYZ')
    
    dice, _ = metrics.dice_coef(pred, gt)
    jaccard = metrics.volume_jaccard_index(pred, gt)
    stats = metrics.segmentation_stats(pred, gt)
    
    results.append({
        'test_name': name,
        'dice': dice,
        'jaccard': jaccard,
        '#cells': stats['total'],
        'correct': stats['correct'],
        'missing': stats['missing'],
        'over_segmented': stats['over_segmented'],
        'under_segmented': stats['under_segmented'],
        'confused': stats['confused']
    })
    
    
    visualization.save_comparison(pred, stats, out_path)
    bar.update()
    
bar.end()

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv(pred_paths + 'Stats/results.csv')